In [1]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd
import dash_daq as daq


# Load the CSV file into a DataFrame
df = pd.read_csv('Student Profiles Wrangled.csv')

C:\Users\user\AppData\Local\Temp\ipykernel_3828\2560784446.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\user\AppData\Local\Temp\ipykernel_3828\2560784446.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
def categorize_gpa(row):
    gpa = row['GPA']
    if gpa < 1:
        return 'Less Than 1'
    elif gpa < 2 and gpa >= 1:
        return '1 - 2'
    elif gpa < 3 and gpa >= 2:
        return '2 - 3'
    elif gpa < 3.5 and gpa >= 3:
        return '3 - 3.5'
    else:
        return 'More Than 3.5'

df['GPA Category'] = df.apply(categorize_gpa, axis=1)

In [3]:
df

,STUDENT ID,SALUTATION,GENDER,NATIONALITY,DOB,HIGHEST QUALIFICATION,NAME OF QUALIFICATION AND INSTITUTION,DATE ATTAINED HIGHEST QUALIFICATION,DESIGNATION,INTAKE NO,...,COURSE FUNDING,REGISTRATION FEE,PAYMENT MODE,COURSE FEE,GPA,CITIZENSHIP_STATUS,date_diff,COURSE,Age,GPA Category
0,2020/1101-013/001,Ms,F,Singaporean,1978-04-03,Certificate,"Certificate in Office Skills, ITE",2016-11-06,"Snr Associate, Client Services",13th,...,Subsidiesed,107,NETS,1712,3.0,L,456,1101,46,3 - 3.5
1,2020/1101-013/002,Ms,F,Singaporean,1966-03-23,Certificate,"WSQ Higher Certificate in Human Resources, WPL...",2018-02-06,Admin Officer,13th,...,Individual,107,NETS,1712,2.5,L,456,1101,58,2 - 3
2,2020/1101-013/003,Ms,F,Singaporean,1988-04-12,Certificate,"Nitec in Service Skills (Office), ITE",2016-11-06,Admin Assistant,13th,...,Sponsored,107,Bank Transfer,1712,2.0,L,456,1101,36,2 - 3
3,2020/1101-014/004,Ms,F,Singaporean,1991-11-27,Degree,Bachelor Science (Facilities & Events Manageme...,2017-06-10,HR Administrator,14th,...,Subsidiesed,107,NETS,1212,2.5,L,456,1101,32,2 - 3
4,2020/1101-014/005,Ms,F,Singaporean,1985-04-23,Diploma,"Diploma in Procurement and Supply Management, ...",2016-08-10,Purchasing Executive,14th,...,Subsidiesed,107,NETS,1712,2.4,L,456,1101,39,2 - 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,2019/5113-007/005,Ms,F,Singaporean,1973-01-10,Degree,Bachelor of Commerce (Accounting and Finance)/...,2017-11-05,"Head, Registry & Corp Admin",7th,...,Individual,107,Bank Transfer,5803,2.9,L,183,5113,51,2 - 3
240,2019/5113-007/006,Ms,F,Permanent Resident,1982-09-30,Degree,Bachelor of Economics (Accounting)/\r\nUnivers...,2016-05-05,Finance Officer,7th,...,Individual,107,Bank Transfer,5803,2.0,L,183,5113,41,2 - 3
241,2019/5113-006/003,Ms,F,Singaporean,1990-06-03,Degree,Bachelor of Science in Hotel Administration (H...,2015-06-08,HR Assistant,6th,...,Individual,107,Bank Transfer,5803,2.8,L,183,5113,34,2 - 3
242,2019/5113-005/001,Ms,F,Singaporean,1981-11-14,Degree,Bachelor of Arts with Second Class Honours (Lo...,2014-08-04,Household Goods Coordinator,5th,...,Individual,107,Bank Transfer,5803,1.9,L,180,5113,42,1 - 2


In [4]:
import pandas as pd
import re

def categorize_job_title(title):
    if pd.isna(title):
        return 'Not Specified'
    
    title = str(title).lower()
    
    if re.search(r'hr|human resource|recruit|talent|people ops', title):
        return 'Human Resources'
    elif re.search(r'admin|secretary|receptionist|clerk|office', title):
        return 'Administration'
    elif re.search(r'manager|director|head|lead|supervisor|vice president', title):
        return 'Management'
    elif re.search(r'account|finance|payroll', title):
        return 'Finance and Accounting'
    elif re.search(r'client|sales|service|customer', title):
        return 'Customer Service and Sales'
    elif re.search(r'operation|logistics|supply|coordinator', title):
        return 'Operations and Logistics'
    elif re.search(r'market|pr|public relation|brand|communication', title):
        return 'Marketing and Public Relations'
    elif re.search(r'teacher|lecturer|trainer|education|learning', title):
        return 'Education and Training'
    elif re.search(r'it|tech|developer|engineer|analyst', title):
        return 'Technical and IT'
    else:
        return 'Others'

def add_job_category(df):
    df['Job Category'] = df['DESIGNATION'].apply(categorize_job_title)
    return df

# Assuming your dataframe is called 'df'
df = add_job_category(df)

# Plotly Sunburst

In [5]:
# Prepare the data for the sunburst plot
# Create simplified age groups
df['Age Group'] = pd.cut(df['Age'], bins=[18, 35, 60], labels=['Millenial', 'Middle Aged'])

# Drop rows with missing values in the relevant columns
df.dropna(subset=['COURSE', 'Age Group', 'GPA'], inplace=True)

# Filter data to include only the top 5 courses by student count
top_courses = df['COURSE'].value_counts().nlargest(5).index
df_filtered = df[df['COURSE'].isin(top_courses)]

# Add a Count column for aggregation
df_filtered['Count'] = 1

# Group by course and age group and calculate average GPA and count
df_grouped = df_filtered.groupby(['COURSE', 'Age Group']).agg(
    Count=('Count', 'sum'),
    Avg_GPA=('GPA', 'mean')
).reset_index()

# Calculate average GPA for each level
course_gpa = df_filtered.groupby('COURSE')['GPA'].mean().reset_index(name='GPA_course')

# Merge the average GPAs into the grouped dataframe
df_grouped = df_grouped.merge(course_gpa, on='COURSE')

# Create the sunburst plot
fig = px.sunburst(df_grouped, path=['COURSE', 'Age Group'], values='Count',
                  color='Avg_GPA', 
                  hover_data={
                      'Avg_GPA': ':.2f',
                      'GPA_course': ':.2f'
                  })

# Update hover template to show average GPAs for each level
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>' +
                  'Count: %{value}<br>' +
                  'Course GPA: %{customdata[1]:.2f}<br>' +
                  'Age Group GPA: %{customdata[0]:.2f}'
)

# Update layout for better aesthetics and readability
fig.update_layout(
    title={
        'text': 'Course and Age with Average GPAs',
        'y':0.975,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='black'),
    font=dict(family='Arial, sans-serif', size=16, color='black'),
    margin=dict(t=40, l=0, r=0, b=10),
    paper_bgcolor='white',
    plot_bgcolor='white'
)

# Show the plot
fig.show()


In [6]:
import pandas as pd
import plotly.express as px
import numpy as np

# Assuming df is your DataFrame
# If not, you need to load your data into df first

# Create simplified age groups
df['Age Group'] = pd.cut(df['Age'], bins=[18, 35, 60], labels=['Millenial', 'Middle Aged'])

# Drop rows with missing values in the relevant columns
df.dropna(subset=['HIGHEST QUALIFICATION', 'Age Group', 'GPA'], inplace=True)

# Filter data to include only the top 5 courses by student count
top_courses = df['HIGHEST QUALIFICATION'].value_counts().nlargest(6).index
df_filtered = df[df['HIGHEST QUALIFICATION'].isin(top_courses)]

# Add a Count column for aggregation
df_filtered['Count'] = 1

# Group by course and age group and calculate average GPA and count
df_grouped = df_filtered.groupby(['HIGHEST QUALIFICATION', 'Age Group']).agg(
    Count=('Count', 'sum'),
    Avg_GPA=('GPA', 'mean')
).reset_index()

# Calculate average GPA for each level
course_gpa = df_filtered.groupby('HIGHEST QUALIFICATION')['GPA'].mean().reset_index(name='GPA_course')

# Merge the average GPAs into the grouped dataframe
df_grouped = df_grouped.merge(course_gpa, on='HIGHEST QUALIFICATION')

# Filter out any potential zero counts
df_grouped = df_grouped[df_grouped['Count'] > 0]

# Create the sunburst plot
fig = px.sunburst(df_grouped, path=['HIGHEST QUALIFICATION', 'Age Group'], values='Count',
                  color='Avg_GPA', 
                  hover_data={
                      'Avg_GPA': ':.2f',
                      'GPA_course': ':.2f'
                  })

# Update hover template to show average GPAs for each level
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>' +
                  'Count: %{value}<br>' +
                  'Course GPA: %{customdata[1]:.2f}<br>' +
                  'Age Group GPA: %{customdata[0]:.2f}'
)

# Update layout for better aesthetics and readability
fig.update_layout(
    title={
        'text': 'Course and Age with Average GPAs',
        'y':0.975,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='black'),
    font=dict(family='Arial, sans-serif', size=16, color='black'),
    margin=dict(t=40, l=0, r=0, b=10),
    paper_bgcolor='white',
    plot_bgcolor='white'
)

# Show the plot
fig.show()

In [7]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

colors = {
    'background': '#111111',
    'text': '#ffffff'
}

df['COURSE'] = df['COURSE'].astype(str)

# Create initial figures
fig_bar = px.bar(df, x="COURSE", barmode="group")
fig_pie_1 = px.pie(df, values='GPA', names='COURSE', title='Course Distribution')
fig_pie_2 = px.pie(df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

# Update layout for all figures
for fig in [fig_bar, fig_pie_1, fig_pie_2]:
    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text']
    )

Age_Range = df['Age'].unique()
Age_Range.sort()

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("DAVI CA2 Dashboard", className="text-center mb-4 mt-4"), width=12)
    ]),

    dbc.Row([
        # Sidebar with radio buttons
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H3("Filters", className="text-center")),
                dbc.CardBody([
                    html.H6("Job Category"),
                    dcc.Dropdown(id='Job-category', options=[{'label': 'All Data', 'value': 'ALL'}]+[{'label': country, 'value': country}
                          for country in df['Job Category'].unique()],style={'color': 'black'}, value='ALL',multi=True),
                    html.H6("Nationality", className='mt-4'),
                    dcc.RadioItems(
                        id='radio-buttons',
                        options=[{'label': 'All Data', 'value': 'ALL'}] + 
                                [{'label': nat, 'value': nat} for nat in df['NATIONALITY'].unique()],
                        value='ALL',
                        className="mb-2"
                    ),
                    html.H6("Age Range"),
                    dcc.RangeSlider(Age_Range.min(), Age_Range.max(), 3, value=[Age_Range.min(), Age_Range.max()], id='my-range-slider'),
                    html.H6("Male or Female"),
                    daq.BooleanSwitch(
                    on=True,
                    color="#9B51E0",
                    )

                ])
            ])
        ], width=3),

        # Main content area
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='example-graph-1', figure=fig_bar)
                ])
            ], className="mb-4"),

            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='example-pie-1', figure=fig_pie_1)
                        ])
                    ])
                ], width=6),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='example-pie-2', figure=fig_pie_2)
                        ])
                    ])
                ], width=6)
            ])
        ], width=9)
    ])
], fluid=True, style={'background-color': 'eggshell'})

# Define callback to update graphs
@app.callback(
    Output('example-graph-1', 'figure'),
    Output('example-pie-1', 'figure'),
    Output('example-pie-2', 'figure'),
    Input('radio-buttons', 'value'),
    Input('Job-category', 'value')
)

def update_graphs(selected_nationality, selected_job_categories):
    # If 'ALL' is selected in either dropdown, it means no filtering on that dimension
    if 'ALL' in selected_nationality:
        filtered_df = df
    else:
        filtered_df = df[df['NATIONALITY'].isin(selected_nationality)]
    
    if 'ALL' not in selected_job_categories:
        filtered_df = filtered_df[filtered_df['Job Category'].isin(selected_job_categories)]
    
    fig_bar = px.bar(filtered_df, x="COURSE", y="GPA", barmode="group",
                     title="GPA Distribution by Course and Gender")
    fig_pie_1 = px.pie(filtered_df, values='GPA', names='COURSE', title='Course Distribution')
    fig_pie_2 = px.pie(filtered_df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

    for fig in [fig_bar, fig_pie_1, fig_pie_2]:
        fig.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
        )

    return fig_bar, fig_pie_1, fig_pie_2
    
if __name__ == '__main__':
    app.run_server(debug=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 242 entries, 0 to 243
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   STUDENT ID                             242 non-null    object  
 1   SALUTATION                             242 non-null    object  
 2   GENDER                                 242 non-null    object  
 3   NATIONALITY                            242 non-null    object  
 4   DOB                                    242 non-null    object  
 5   HIGHEST QUALIFICATION                  242 non-null    object  
 6   NAME OF QUALIFICATION AND INSTITUTION  242 non-null    object  
 7   DATE ATTAINED HIGHEST QUALIFICATION    242 non-null    object  
 8   DESIGNATION                            242 non-null    object  
 9   INTAKE NO                              242 non-null    object  
 10  COMMENCEMENT DATE                      242 non-null    object  
 11

In [9]:
df.GENDER

0      F
1      F
2      F
3      F
4      F
      ..
239    F
240    F
241    F
242    F
243    F
Name: GENDER, Length: 242, dtype: object

In [10]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import dash_daq as daq

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

colors = {
    'background': '#ffffff',
    'text': '#333333',
    'primary': '#007bff'
}

fig_bar = px.bar(df, x="COURSE")
fig_pie_1 = px.pie(df, values='GPA', names='COURSE', title='Course Distribution')
fig_pie_2 = px.pie(df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

# Update layout for all figures
for fig in [fig_bar, fig_pie_1, fig_pie_2]:
    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text']
    )

Age_Range = df['Age'].unique()
Age_Range.sort()

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
                html.H1("Rejeys Dashboard", className="mb-4 mt-4 text-align-center")], width=12,class_name='text-center')
    ]),

    dbc.Row([
        # Sidebar
        dbc.Col([
            html.H4("DAVI CA2 Dashboard", className="mb-3 mt-3"),
            html.H6("Job Category"),
            dcc.Dropdown(
                id='Job-category',
                options=[{'label': 'All Data', 'value': 'ALL'}] + [{'label': country, 'value': country} for country in df['Job Category'].unique()],
                value='ALL',
                multi=True,
                style={'color': colors['text']}
            ),
            html.H6("Nationality", className='mt-4'),
            dcc.Dropdown(
                id='radio-buttons',
                options=[{'label': 'All Data', 'value': 'ALL'}] + [{'label': nat, 'value': nat} for nat in df['NATIONALITY'].unique()],
                value='ALL',
                style={'color': colors['text']}
            ),
            html.H6("Age Range", className='mt-4'),
            dcc.RangeSlider(Age_Range.min(), Age_Range.max(), 3, value=[Age_Range.min(), Age_Range.max()], id='my-range-slider'),
            html.H6("Gender", className='mt-4'),
            dcc.RadioItems(['ALL','M','F'],value='ALL',id='Gener')

        ], width=3, className="bg-light"),

        # Main content area
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='example-graph-1', figure=fig_bar)
                ])
            ], className="mb-4"),

            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='example-pie-1', figure=fig_pie_1)
                        ])
                    ])
                ], width=6),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='example-pie-2', figure=fig_pie_2)
                        ])
                    ])
                ], width=6)
            ])
        ], width=9)
    ])
], fluid=True, style={'background-color': colors['background']})

# Keep your existing callback function

@app.callback(
    Output('example-graph-1', 'figure'),
    Output('example-pie-1', 'figure'),
    Output('example-pie-2', 'figure'),
    Input('radio-buttons', 'value'),
    Input('Job-category', 'value'),
    Input('my-range-slider', 'value'),
    Input('Gener', 'value')
)
def update_graphs(selected_nationality, selected_job_categories, age_range, selected_gender):
    # Start with the full dataset
    filtered_df = df

    # Filter by nationality
    if selected_nationality != 'ALL':
        filtered_df = filtered_df[filtered_df['NATIONALITY'] == selected_nationality]
    
    # Filter by job category
    if 'ALL' not in selected_job_categories:
        filtered_df = filtered_df[filtered_df['Job Category'].isin(selected_job_categories)]
    
    # Filter by age range
    filtered_df = filtered_df[(filtered_df['Age'] >= age_range[0]) & (filtered_df['Age'] <= age_range[1])]
    
    # Filter by gender
    if selected_gender != 'ALL':
        filtered_df = filtered_df[filtered_df['GENDER'] == selected_gender]
    
    # Create updated figures
    fig_bar = px.bar(filtered_df, x="COURSE", y="GPA", color="GENDER",
                     title="GPA Distribution by Course and Gender")
    fig_pie_1 = px.pie(filtered_df, values='GPA', names='COURSE', title='Course Distribution')
    fig_pie_2 = px.pie(filtered_df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

    # Update layout for all figures
    for fig in [fig_bar, fig_pie_1, fig_pie_2]:
        fig.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
        )

    return fig_bar, fig_pie_1, fig_pie_2
    
if __name__ == '__main__':
    app.run_server(debug=True)

In [14]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import dash_daq as daq

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

colors = {
    'background': '#f8f9fa',
    'text': '#2c3e50',
    'primary': '#3498db',
    'secondary': '#2ecc71',
    'accent': '#e74c3c'
}

# Assuming df is your DataFrame
fig_bar = px.bar(df, x="COURSE")
fig_pie_1 = px.pie(df, values='GPA', names='COURSE', title='Course Distribution')
fig_pie_2 = px.pie(df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

# Update layout for all figures
for fig in [fig_bar, fig_pie_1, fig_pie_2]:
    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        title_font_size=20,
        legend_title_font_size=14,
        legend_font_size=12,
        margin=dict(l=40, r=40, t=40, b=40)
    )
    fig.update_traces(marker=dict(line=dict(color='#ffffff', width=2)))

Age_Range = df['Age'].unique()
Age_Range.sort()

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Rejey DAVI Dashboard", className="display-4 text-primary mb-4")
        ], width=12, className='text-center')
    ]),

    dbc.Row([
        # Sidebar
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Filters", className="card-title text-primary mb-4"),
                    html.H6("Job Category", className="text-muted"),
                    dcc.Dropdown(
                        id='Job-category',
                        options=[{'label': 'All Data', 'value': 'ALL'}] + [{'label': country, 'value': country} for country in df['Job Category'].unique()],
                        value='ALL',
                        multi=True,
                        style={'color': colors['text']},
                        className="mb-3"
                    ),

                    html.H6("Nationality", className='text-muted mt-4'),
                    dcc.Dropdown(
                        id='radio-buttons',
                        options=[{'label': 'All Data', 'value': 'ALL'}] + [{'label': nat, 'value': nat} for nat in df['NATIONALITY'].unique()],
                        value='ALL',
                        style={'color': colors['text']},
                        className="mb-3"
                    ),
                    html.H6("Age Range", className='text-muted mt-4'),
                    dcc.RangeSlider(Age_Range.min(), Age_Range.max(), 3, value=[Age_Range.min(), Age_Range.max()], id='my-range-slider',
                                    marks={i: str(i) for i in range(int(Age_Range.min()), int(Age_Range.max())+1, 5)},
                                    className="mb-3"),
                    html.H6("Gender", className='text-muted mt-4'),
                    dbc.RadioItems(
                        id='Gender',
                        options=[
                            {'label': 'All', 'value': 'ALL'},
                            {'label': 'Male', 'value': 'M'},
                            {'label': 'Female', 'value': 'F'}
                        ],
                        value='ALL',
                        inline=True,
                        className="mb-3"
                    )
                ])
            ], className="mb-4 shadow-sm", style={'border-radius': '10px', 'height': '92%'})  # Ensure full height
        ], width=3),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='example-graph-1', figure=fig_bar)
                ])
            ], className="mb-4"),
            
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='example-pie-1', figure=fig_pie_1)
                        ])
                    ])
                ], width=6),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='example-pie-2', figure=fig_pie_2)
                        ])
                    ])
                ], width=6)
            ])
        ], width=9)
    ])
], fluid=True, style={})

@app.callback(
    Output('example-graph-1', 'figure'),
    Output('example-pie-1', 'figure'),
    Output('example-pie-2', 'figure'),
    Input('radio-buttons', 'value'),
    Input('Job-category', 'value'),
    Input('my-range-slider', 'value'),
    Input('Gender', 'value')
)
def update_graphs(selected_nationality, selected_job_categories, age_range, selected_gender):
    filtered_df = df

    if selected_nationality != 'ALL':
        filtered_df = filtered_df[filtered_df['NATIONALITY'] == selected_nationality]
    
    if 'ALL' not in selected_job_categories:
        filtered_df = filtered_df[filtered_df['Job Category'].isin(selected_job_categories)]
    
    filtered_df = filtered_df[(filtered_df['Age'] >= age_range[0]) & (filtered_df['Age'] <= age_range[1])]
    
    if selected_gender != 'ALL':
        filtered_df = filtered_df[filtered_df['GENDER'] == selected_gender]
    
    fig_bar = px.bar(filtered_df, x="COURSE", y="GPA", color="GENDER",
                     title="GPA Distribution by Course and Gender",
                     color_discrete_map={'M': colors['primary'], 'F': colors['secondary']})
    fig_pie_1 = px.pie(filtered_df, values='GPA', names='COURSE', title='Course Distribution')
    fig_pie_2 = px.pie(filtered_df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

    for fig in [fig_bar, fig_pie_1, fig_pie_2]:
        fig.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text'],
            title_font_size=20,
            legend_title_font_size=14,
            legend_font_size=12,
            margin=dict(l=40, r=40, t=40, b=40)
        )
        fig.update_traces(marker=dict(line=dict(color='#ffffff', width=2)))

    return fig_bar, fig_pie_1, fig_pie_2

if __name__ == '__main__':
    app.run_server(debug=True)
